In [9]:
import torch
import numpy as np
import os,sys,os.path
from tensorboardX import SummaryWriter
import pickle
from torch import nn
import hashlib

In [10]:
from models import CNNFemnist,ResNet18,ShuffLeNet
from sampling import LocalDataset, LocalDataloaders, partition_data
from option import args_parser

from Server.ServerFedAvg import ServerFedAvg
from Server.ServerFedProx import ServerFedProx
from Server.ServerFedMD import ServerFedMD 
from Server.ServerFedProto import ServerFedProto
from Server.ServerFedDFKD import ServerFedDFKD

In [11]:
torch.set_default_dtype(torch.float64)
print(torch.__version__)
torch.cuda.is_available()
device = torch.device("cuda:0")
args = args_parser()
np.set_printoptions(threshold=np.inf)
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3,4,5,6,7'

print(args)
args_hash = ''
for k,v in vars(args).items():
    if k == 'eval_only':
        continue
    args_hash += str(k)+str(v)
    
args_hash = hashlib.sha256(args_hash.encode()).hexdigest()

1.9.0+rocm4.2


usage: ipykernel_launcher.py [-h] [--dataset DATASET] [--log_frq LOG_FRQ]
                             [--batch_size BATCH_SIZE]
                             [--num_epochs NUM_EPOCHS] [--clip_grad CLIP_GRAD]
                             [--lr LR] [--lr_sh_rate LR_SH_RATE]
                             [--use_lrschd] [--num_clients NUM_CLIENTS]
                             [--num_classes NUM_CLASSES]
                             [--sampling_rate SAMPLING_RATE]
                             [--local_ep LOCAL_EP] [--beta BETA] [--seed SEED]
                             [--code_len CODE_LEN] [--alg ALG] [--lam LAM]
                             [--gamma GAMMA] [--std STD] [--part PART]
                             [--sample SAMPLE] [--temp TEMP] [--model MODEL]
                             [--upload_model] [--eval_only]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/huancheng/.local/share/jupyter/runtime/kernel-c211f2ff-2dcb-4a24-9745-c6d05f9357eb.json


SystemExit: 2

In [ ]:
train_dataset,testset, dict_users, dict_users_test = partition_data(n_users = args.num_clients, alpha=args.beta,rand_seed = args.seed, dataset=str(args.dataset))

In [ ]:
Loaders_train = LocalDataloaders(train_dataset,dict_users,args.batch_size,ShuffleorNot = True,frac=args.part)
Loaders_test = LocalDataloaders(testset,dict_users_test,args.batch_size,ShuffleorNot = True,frac=args.part)
global_loader_test = torch.utils.data.DataLoader(testset, batch_size=args.batch_size,shuffle=True, num_workers=2)

for idx in range(args.num_clients):
    counts = [0]*args.num_classes
    for batch_idx,(X,y) in enumerate(Loaders_train[idx]):
        batch = len(y)
        y = np.array(y)
        for i in range(batch):
            counts[int(y[i])] += 1
    print('Client {} data distribution:'.format(idx))
    print(counts)

In [4]:
logger = SummaryWriter('./logs')
checkpoint_dir = './checkpoint/'+ args.dataset + '/'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
with open(checkpoint_dir+'args.pkl', 'wb') as fp:
    pickle.dump(args, fp)
print('Checkpoint dir:', checkpoint_dir)

NameError: name 'args' is not defined

In [5]:
if args.model == 'CNN':
    global_model = CNNFemnist(args, code_length=args.code_len, num_classes = args.num_classes)
    
if args.model == 'resnet18':
    global_model = ResNet18(args, code_length=args.code_len, num_classes = args.num_classes)

if args.model == 'shufflenet':  
    global_model = ShuffLeNet(args, code_length=args.code_len, num_classes = args.num_classes)

   
print('# model parameters:', sum(param.numel() for param in global_model.parameters()))
global_model = nn.DataParallel(global_model)
global_model.to(device)

NameError: name 'args' is not defined

In [6]:
if args.alg == 'FedAvg':
    server = ServerFedAvg(args,global_model,Loaders_train,Loaders_test,global_loader_test,logger,device)
if args.alg == 'FedProx':
    server = ServerFedProx(args,global_model,Loaders_train,Loaders_test,global_loader_test,logger,device)
if args.alg == 'FedMD':
    server = ServerFedMD(args,global_model,Loaders_train,Loaders_test,global_loader_test,testset,logger,device)
if args.alg == 'FedProto':    
    server = ServerFedProto(args,global_model,Loaders_train,Loaders_test,global_loader_test,logger,device)
if args.alg == 'FedDFKD':    
    server = ServerFedDFKD(args,global_model,Loaders_train,Loaders_test,global_loader_test,logger,device)

NameError: name 'args' is not defined

In [7]:
server.Create_Clints()

NameError: name 'server' is not defined

In [8]:
server.train()

NameError: name 'server' is not defined

In [ ]:
server.global_test_accuracy()

In [17]:
save_path = checkpoint_dir + args_hash + '.pth'

In [19]:
if args.upload_model == True:
    server.Save_CheckPoint(save_path)
    print('Model is saved on: ')
    print(save_path)